In [ ]:
import pandas as pd
import os
import numpy as np

# <b>1. 민원 데이터 파일 이어붙이기</b>

In [ ]:
# 파일 목록 불러오기
# 파일 폴더 경로 지정
PATH = 'C:\민원분석참조모델\data\민원'
os.chdir(PATH)
df_list = list(os.listdir(os.getcwd()))
df_list

In [ ]:
# 이름 지정하기
name = '20190101-20201231.csv'

In [ ]:
# 파일 이어붙이기
df_merged = pd.DataFrame()
for i in range(len(df_list)):
    df = pd.read_csv(df_list[i], encoding='euc-kr')
    df_merged = pd.concat([df_merged, df], ignore_index=True)

In [ ]:
# 파일 하나 전처리할 시
df = pd.read_csv(df_list[0], encoding='cp949')
df.to_csv(f'./최종_{df_list[0]}', encoding='cp949', index=False)

In [ ]:
# 병합 파일 저장하기
df_merged.to_csv(f'../결과/{name}', encoding='cp949', index=False)

---

# <b>2. 필요한 컬럼 가져오기</b>

In [ ]:
# 최종 파일 불러오기
df = pd.read_csv(f'../결과/{name}', encoding='cp949')
# df = pd.read_csv(f'./최종_{df_list[0]}', encoding='euc-kr')

In [ ]:
# 파일 개요
df.info()

In [ ]:
# 파일 결측값 제거
df = df.dropna()
df.info()

In [ ]:
# 컬럼명 좌우 공백 제거
for i in range(len(df.columns)):
    df.rename(columns={df.columns[i]: df.columns[i].strip()})

In [ ]:
# 필요한 컬럼 가져오기
df = df[['민원접수번호', '민원등록일자', '민원등록일시', '민원제목', '민원내용',
        '민원처리기한', '담당자지정일시', '부서코드',
         '처리부서명', '처리담당자명', '답변처리일시', '민원인 주소',
         '연장처리일수 ', '연장처리횟수', '연장처리기한', '실처리일수']]
df.columns

In [ ]:
# 컬럼명 변경
df = df.rename(columns={
    '민원처리기한': '처리기한일시',
    '처리부서명': '처리부서',
    '처리담당자명': '처리담당자',
    '답변처리일시': '답변일시',
    '민원인 주소': '민원인주소',
    '연장처리일수 ': '연장처리일수'})
df.columns

---

# <b>3. R 새올 민원 분석 시 - 민원접수번호 전처리</b>

In [ ]:
df['민원접수번호']

In [ ]:
# 민원접수번호 object -> float 변경 
df['민원접수번호'] = df['민원접수번호'].astype('float')

In [ ]:
# 민원접수번호 float -> int
df['민원접수번호'] = df['민원접수번호'].astype('int64')

In [ ]:
# 민원접수번호 int -> str 변경
df['민원접수번호'] = df['민원접수번호'].astype('str')

In [ ]:
# 민원접수번호에 ' 문자열 추가
for i in df.index:
    df['민원접수번호'][i] = '\'' + df['민원접수번호'][i]

---

# <b>4. R 새올 민원 분석 시 - 일시 전처리</b>

In [ ]:
df['민원등록일시'] = pd.to_datetime(df['민원등록일시']).dt.strftime('%Y-%m-%d %H:%M')
df['처리기한일시'] = pd.to_datetime(df['처리기한일시']).dt.strftime('%Y-%m-%d %H:%M')
df['담당자지정일시'] = pd.to_datetime(df['담당자지정일시']).dt.strftime('%Y-%m-%d %H:%M')
df['답변일시'] = pd.to_datetime(df['답변일시']).dt.strftime('%Y-%m-%d %H:%M')

---

# <b>4. R 새올 민원 분석 시 - Nan 전처리</b>

In [ ]:
# float 변경
df['연장처리일수'] = df['연장처리일수'].astype('float')
df['연장처리횟수'] = df['연장처리횟수'].astype('float')
df['실처리일수'] = df['실처리일수'].astype('float')

In [ ]:
# 0인 값 nan 처리
df['연장처리일수'].replace(0, np.nan, inplace=True)
df['연장처리일수'].unique()

In [ ]:
# 0인 값 nan 처리
df['연장처리횟수'].replace(0, np.nan, inplace=True)
df['연장처리횟수'].unique()

In [ ]:
# 실처리일수 유일치 확인
df['실처리일수'].replace(0, np.nan, inplace=True)
df['실처리일수'].unique()

In [ ]:
# 연장처리기한 유일치 확인
df['연장처리기한'].unique()

In [ ]:
# 0인 값 nan 처리
df['연장처리기한'].replace(" ", np.nan, inplace=True)
df['연장처리기한'].unique()

In [ ]:
# 연장처리기한 포맷
df['연장처리기한'] = pd.to_datetime(df['연장처리기한']).dt.strftime('%Y-%m-%d %H:%M')
df['연장처리기한'].unique()

In [ ]:
df.head(2)

In [ ]:
# 최종본 파일 저장 - 같은 파일에 덮어쓰기 저장됨
df.to_csv(f'../결과/R_{name}', encoding='cp949', index=False)
# df.to_csv(f'./최종파일_{df_list[0]}', encoding='cp949', index=False)